In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import minmax_scale
from sklearn.cluster import KMeans

%matplotlib inline
sns.set()

In [ ]:
df_input = pd.read_csv('./enter csv file name')

#	Details	Comp	Tech	Country	ExtraInfo	DL	UL	CP	PP

df_input = df_input[['Details', 'Comp','Tech','Country','ExtraInfo','DL','UL', 'CP','PP']]
#only for US 
df_input =df_input[df_input.Country=='US']
df_input.head()

In [ ]:
df_input.Details.value_counts()
df_input.Tech.value_counts()

In [ ]:
#intermediate df for processing
dataframe = df_input

print('Shape : ',  dataframe.shape)
dataframe.head()

In [ ]:
# Normalize and add normalized PP column
dataframe['PP_norm'] = minmax_scale(dataframe.PP)

In [ ]:
def calculate_cluster(dataframe):
    import random
    cluster=1
    diff=20

    #diff between max and min PPs (range) within a cluster -> lower range => more cluster, higher range => less cluster

    maxdiffwithincluster=8 

    while(diff> maxdiffwithincluster):
        random.seed(100)
        kmeans = KMeans(cluster)
        dataframe['kmeans'] = kmeans.fit_predict(dataframe[['PP_norm']])
        for i in range(cluster):
            local_max=dataframe.loc[dataframe.kmeans==i,'PP'].max()
            local_min=dataframe.loc[dataframe.kmeans==i,'PP'].min()
            diff=local_max-local_min
            if(diff> maxdiffwithincluster):
                cluster=cluster+1
                break
    return dataframe,cluster;

expected output from K-means clustering - record has been classified which cluster it belongs to, example shown below

<img src="Kmeans Clustering.jpg" width="800" height="400">

In [ ]:
def assign_cluster(dataframe,cluster):
    #assign new cluster PPs to a group - for now we PP cluster at max PP for that cluster, 
    #we can introduce new pricing methodologies here
    diff=0
    dataframe['ClusterPP']=0
    for i in range(cluster):
        local_max=dataframe.loc[dataframe.kmeans==i,'PP'].max()
        local_min=dataframe.loc[dataframe.kmeans==i,'PP'].min()
        range_lim= local_max-local_min
        if(range_lim > diff):
            diff=range_lim
        #display max diff in PPs in any cluster
        #print(range_lim)
        #assign new cluster PPs
        dataframe.loc[dataframe.kmeans==i,'ClusterPP'] =local_max
    return dataframe,diff;

In [ ]:
def print_cluster_details(dataframe,cluster,diff):
    print("Total plans in this cluster =",len(dataframe.Details))
    print("After clustering no of plans =",cluster)
    print("Average plans per cluster =", round(len(dataframe.Details)/cluster,2))
    print("Max range within a cluster =",diff)

In [ ]:
def cluster_details(dataframe,unique_plans):
    combine_df = pd.DataFrame(columns=['Plan','Cluster','PP','Count'])
    clusterPP=0
    for i in unique_plans:
        df=dataframe[ dataframe.Cluster.str.endswith(i) ]
        #print(df)
        concatDetails=''
        for j in range(len(df)):
            dl=df.iloc[j]['DL'].astype('str')
            if(len(dl)>8):
                dl="{:.3f}".format(float(dl))
            ul=df.iloc[j]['UL'].astype('str')
            if(len(ul)>8):
                ul="{:.3f}".format(float(ul))
            try:
                extraInfo=df.iloc[0]['ExtraInfo']
                if(np.isnan(extraInfo)):
                    extraInfo=''
            except:
                extraInfo=' '+extraInfo
            concatDetails=concatDetails +df.iloc[j]['Tech']+ ' by '+ df.iloc[j]['Comp'] + ' '+ dl+ 'M/'+ ul +'M' + extraInfo+ '\n'
            clusterPP=df.iloc[j]['ClusterPP']
        combine_df=combine_df.append({'Plan': concatDetails,'Cluster': i,'PP': clusterPP, 'Count': j+1}, ignore_index=True)
        break
    return(combine_df)

In [ ]:
def cluster_moreDetails(dataframe,unique_plans):
    combine_df = pd.DataFrame(columns=['Plan','Cluster','PP','Count','Cluster_Def','Reduced_Count'])
    clusterPP=0
    for i in unique_plans:
        df=dataframe[ dataframe.Cluster.str.endswith(i) ]
        data=
        {
            'Comp':df.groupby(['Comp','Tech'], as_index=False)['DL'].min()['Comp'],
            'Tech':df.groupby(['Comp','Tech'], as_index=False)['DL'].min()['Tech'],
            'DL-min':df.groupby(['Comp','Tech'], as_index=False)['DL'].min()['DL'],
            'UL-min':df.groupby(['Comp','Tech'], as_index=False)['UL'].min()['UL'],
            'DL-max':df.groupby(['Comp','Tech'], as_index=False)['DL'].max()['DL'],
            'UL-max':df.groupby(['Comp','Tech'], as_index=False)['UL'].max()['UL']
        }
        temp_df = pd.DataFrame(data)
        summariseDetails=''
        
        for k in range(len(temp_df)):
            dl_summ=''; ul_summ=''
            dl_min='' ; dl_max=''
            ul_min='' ; ul_max=''
            
            dl_min = temp_df.iloc[k]['DL-min'].astype('str')
            if(len(dl_min)>8):
                dl_min="{:.3f}".format(float(dl_min))
            
            dl_max = temp_df.iloc[k]['DL-max'].astype('str')
            if(len(dl_max)>8):
                dl_max="{:.3f}".format(float(dl_max))
                
            ul_min = temp_df.iloc[k]['UL-min'].astype('str')
            if(len(ul_min)>8):
                ul_min="{:.3f}".format(float(ul_min))
            
            ul_max = temp_df.iloc[k]['UL-max'].astype('str')
            if(len(ul_max)>8):
                ul_max="{:.3f}".format(float(ul_max))
                
            if(temp_df['DL-min'][k]==temp_df['DL-max'][k]):
                dl_summ= dl_min + 'M'
            else:
                dl_summ= dl_min + 'M-' + dl_max + 'M' 
            
            if(temp_df['UL-min'][k]==temp_df['UL-max'][k]):
                ul_summ= ul_min + 'M'
            else:
                ul_summ= ul_min + 'M-' + ul_max + 'M' 
            
            summariseDetails=summariseDetails + temp_df.iloc[k]['Tech']+ ' by '+ temp_df.iloc[k]['Comp'] + ' '+ dl_summ+ ' / '+ ul_summ + '\n'
         
        concatDetails=''
        
        for j in range(len(df)):
            dl=df.iloc[j]['DL'].astype('str')
            if(len(dl)>8):
                dl="{:.3f}".format(float(dl))
            ul=df.iloc[j]['UL'].astype('str')
            if(len(ul)>8):
                ul="{:.3f}".format(float(ul))
            try:
                extraInfo=df.iloc[0]['ExtraInfo']
                if(np.isnan(extraInfo)):
                    extraInfo=''
            except:
                extraInfo=' '+extraInfo
            concatDetails=concatDetails +df.iloc[j]['Tech']+ ' by '+ df.iloc[j]['Comp'] + ' '+ dl+ 'M/'+ ul +'M' + extraInfo+ '\n'
            clusterPP=df.iloc[j]['ClusterPP']
            #print(concatDetails)
        combine_df=combine_df.append({'Plan': concatDetails,'Cluster': i,'PP': clusterPP, 'Count': j+1,'Cluster_Def': summariseDetails,'Reduced_Count':k+1}, ignore_index=True)
        
    return(combine_df)

In [ ]:
def clean_data(dataframe):
    #Rename column and delete PP_norm variable
    #	Details	DL	UL	CP	PP	PP_norm	kmeans
    dataframe.rename(columns={'kmeans':'Cluster'}, inplace=True)
    del dataframe['PP_norm']
    return(dataframe)

In [ ]:
def write_to_file(dataframe,combine_df):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    import datetime
    t1 = datetime.datetime.now()
    filename=r'enter file name'+str(t1.hour)+"__"+str(t1.minute)+str(t1.microsecond)+".xlsx"
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')

    # Write each dataframe to a different worksheet.
    dataframe.to_excel(writer, sheet_name='clusters')
    combine_df.to_excel(writer, sheet_name='clustersCombined')

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

In [ ]:
final_details_df=pd.DataFrame()
combine_details_df=pd.DataFrame()

unique_plans = set(df_input.Tech)

for i in unique_plans:
    df_temp = dataframe[ df_input.Tech.str.endswith(i) ]

    # Remove weird plans
    df_temp = df_temp[df_temp.DL >= df_temp.UL]

    df_temp,cluster=calculate_cluster(df_temp)
    
    df_temp,diff=assign_cluster(df_temp,cluster)

    print_cluster_details(df_temp,cluster,diff)

    df_temp=clean_data(df_temp)
    
    df_temp['Cluster']= i +'___' + df_temp['Cluster'].map(str)
    
    final_details_df = final_details_df.append(df_temp)
    
    unique_clusters= set(final_details_df.Cluster)
    
    combine_details_df = cluster_moreDetails(final_details_df,unique_clusters)
       
write_to_file(final_details_df,combine_details_df)

In [ ]:
plt.figure(figsize=(5,5))
sns.pairplot(final_details_df[['DL', 'UL', 'ClusterPP']], hue='ClusterPP', palette='Set1')
plt.show()